In [1]:
from OFA_mbv3_extended.networks.nets.dynamic_nets.single_exit_ofa_mbv3 import *
from OFA_mbv3_extended.networks.nets.dynamic_nets.early_exit_ofa_mbv3 import *
from pprint import pprint
import torch

In [2]:
n_classes=200
bn_param=(0.1, 1e-5)
dropout_rate=0.1
width_mult= 1.0
ks_list=[3,5,7]
expand_ratio_list=[3,4,6]
depth_list=[2,3,4]
net_depth_list=[1,2,3,4,5]
net_width_list=[1,2,3]

# Single_Exit_OFAMobileNetV3

### SE_B_OFAMobileNetV3

In [36]:
base_dyn_net = SE_B_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
)

In [4]:
print(base_dyn_net.name)

SE_B_OFAMobileNetV3


In [5]:
print(base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
(SE(O40, E6.0, K7), None)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
((O80, E6.0, K7), None)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
(SE(O112, E6.0, K7), None)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E6.0, K7), None)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [6]:
print(base_dyn_net.head_stage)

StaticHeadStage(
  (first_conv): ConvLayer(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): Hswish()
  )
  (first_layer): ResidualBlock(
    (conv): MBConvLayer(
      (depth_conv): Sequential(
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (point_linear): Sequential(
        (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (shortcut): IdentityLayer()
  )
)


In [7]:
print(base_dyn_net.exit_stage)

StaticExitStage(
  (final_expand_layer): ConvLayer(
    (conv): Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): Hswish()
  )
  (global_avg_pooling): MyGlobalAvgPool2d(keep_dim=True)
  (feature_mix_layer): ConvLayer(
    (conv): Conv2d(960, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (act): Hswish()
  )
  (classifier): LinearLayer(
    (dropout): Dropout(p=0.1, inplace=True)
    (linear): Linear(in_features=1280, out_features=200, bias=True)
  )
)


In [8]:
base_dyn_net.get_bn_param()

{'momentum': 0.1, 'eps': 1e-05, 'ws_eps': None}

In [9]:
base_dyn_net.set_bn_param(momentum=0.4,eps=2)

In [10]:
base_dyn_net.get_bn_param()

{'momentum': 0.4, 'eps': 2, 'ws_eps': None}

In [11]:
print(base_dyn_net)

SE_B_OFAMobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): DynamicBodyStage(
      (layers): ModuleList(
        (0): Res

In [12]:
net_cfg = base_dyn_net.config

In [13]:
pprint(net_cfg)

{'bn': {'eps': 2, 'momentum': 0.4, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [16],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'DynamicMBConvLayer',
                                       'out_channel_list': [24],
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [24],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'DynamicMBConvLayer',
    

In [14]:
print(base_dyn_net.width_mult)

1.0


In [15]:
print(base_dyn_net.ks_list)

[3, 5, 7]


In [16]:
pprint(base_dyn_net.sample_active_subnet())

{'d': [3, 2, 3, 4, 4],
 'e': [4, 4, 4, 4, 6, 4, 3, 4, 4, 6, 4, 3, 6, 6, 3, 6, 4, 6, 4, 4],
 'ks': [3, 3, 7, 7, 5, 5, 5, 3, 5, 7, 5, 3, 5, 5, 5, 7, 7, 3, 7, 7]}


In [17]:
active_subnet_cfg = base_dyn_net.get_active_net_config()
pprint(active_subnet_cfg)

{'bn': {'eps': 2, 'momentum': 0.4, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'in_channels': 16,
                                       'kernel_size': 3,
                                       'mid_channels': 64,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'in_channels': 24,
                                       'kernel_size': 3,
                                       'mid_channels': 96,
                                

In [18]:
active_subnet = base_dyn_net.get_active_subnet()
pprint(active_subnet.config)

{'bn': {'eps': 2, 'momentum': 0.4, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'groups': None,
                                       'in_channels': 16,
                                       'kernel_size': 3,
                                       'mid_channels': 64,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'groups': None,
                                       'in_channels': 24,
                                      

In [19]:
print(active_subnet)

SE_B_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticBodyStage(
      (layers): ModuleList(
        (0): Residua

In [20]:
print(active_subnet.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv4_RELU_O24_BN, None)
(3x3_MBConv4_RELU_O24_BN, Identity)
(7x7_MBConv4_RELU_O24_BN, Identity)
(SE_5x5_MBConv6_RELU_O40_BN, None)
(SE_5x5_MBConv4_RELU_O40_BN, Identity)
(5x5_MBConv4_H_SWISH_O80_BN, None)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(5x5_MBConv4_H_SWISH_O80_BN, Identity)
(SE_5x5_MBConv6_H_SWISH_O112_BN, None)
(SE_5x5_MBConv6_H_SWISH_O112_BN, Identity)
(SE_5x5_MBConv3_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, None)
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [21]:
print(base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E4.0, K3), None)
((O24, E4.0, K3), Identity)
((O24, E4.0, K7), Identity)
(SE(O40, E6.0, K5), None)
(SE(O40, E4.0, K5), Identity)
((O80, E4.0, K5), None)
((O80, E6.0, K7), Identity)
((O80, E4.0, K5), Identity)
(SE(O112, E6.0, K5), None)
(SE(O112, E6.0, K5), Identity)
(SE(O112, E3.0, K5), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E4.0, K7), None)
(SE(O160, E6.0, K3), Identity)
(SE(O160, E4.0, K7), Identity)
(SE(O160, E4.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [22]:
act_subnet = base_dyn_net.get_active_subnet()

In [23]:
print(act_subnet.module_str)
#should be full

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv4_RELU_O24_BN, None)
(3x3_MBConv4_RELU_O24_BN, Identity)
(7x7_MBConv4_RELU_O24_BN, Identity)
(SE_5x5_MBConv6_RELU_O40_BN, None)
(SE_5x5_MBConv4_RELU_O40_BN, Identity)
(5x5_MBConv4_H_SWISH_O80_BN, None)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(5x5_MBConv4_H_SWISH_O80_BN, Identity)
(SE_5x5_MBConv6_H_SWISH_O112_BN, None)
(SE_5x5_MBConv6_H_SWISH_O112_BN, Identity)
(SE_5x5_MBConv3_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, None)
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [24]:
ks=3
e=[3,3,3,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[2,3,4,4,3]
base_dyn_net.set_active_subnet(ks,e,d)

In [25]:
activated = base_dyn_net.get_active_subnet()
print(activated.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv3_RELU_O24_BN, None)
(3x3_MBConv3_RELU_O24_BN, Identity)
(SE_3x3_MBConv4_RELU_O40_BN, None)
(SE_3x3_MBConv4_RELU_O40_BN, Identity)
(SE_3x3_MBConv4_RELU_O40_BN, Identity)
(3x3_MBConv3_H_SWISH_O80_BN, None)
(3x3_MBConv4_H_SWISH_O80_BN, Identity)
(3x3_MBConv6_H_SWISH_O80_BN, Identity)
(3x3_MBConv6_H_SWISH_O80_BN, Identity)
(SE_3x3_MBConv6_H_SWISH_O112_BN, None)
(SE_3x3_MBConv3_H_SWISH_O112_BN, Identity)
(SE_3x3_MBConv4_H_SWISH_O112_BN, Identity)
(SE_3x3_MBConv4_H_SWISH_O112_BN, Identity)
(SE_3x3_MBConv6_H_SWISH_O160_BN, None)
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [26]:
ks=[3,5,7,5, 5,7,3,5, 5,3,5,7, 7,3,3,5, 7,5,5,7]
e=[3,3,3,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[2,3,4,4,3]
base_dyn_net.set_active_subnet(ks,e,d)

In [37]:
activated = base_dyn_net.get_active_subnet()
print(activated.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, None)
(7x7_MBConv6_RELU_O24_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, Identity)
(SE_7x7_MBConv6_RELU_O40_BN, None)
(SE_7x7_MBConv6_RELU_O40_BN, Identity)
(SE_7x7_MBConv6_RELU_O40_BN, Identity)
(SE_7x7_MBConv6_RELU_O40_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, None)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, None)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O160_BN, None)
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [28]:
base_dyn_net.set_max_net()
print(base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
(SE(O40, E6.0, K7), None)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
((O80, E6.0, K7), None)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
(SE(O112, E6.0, K7), None)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E6.0, K7), None)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [38]:
x = torch.rand((1,3,64,64))
x = activated(x)

### SE_D_OFAMobileNetV3

In [29]:
dense_base_dyn_net = SE_D_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
)

In [30]:
print(dense_base_dyn_net.name)

SE_D_OFAMobileNetV3


In [31]:
print(dense_base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
(SE(O40, E6.0, K7), None)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
((O80, E6.0, K7), None)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
(SE(O112, E6.0, K7), None)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E6.0, K7), None)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [32]:
print(dense_base_dyn_net)

SE_D_OFAMobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): DynamicDenseBodyStage(
      (layers): ModuleList(

In [33]:
net_cfg = dense_base_dyn_net.config

In [34]:
pprint(net_cfg)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [16],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'DynamicMBConvLayer',
                                       'out_channel_list': [24],
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [24],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'DynamicMBConvLayer',


In [35]:
pprint(dense_base_dyn_net.sample_active_subnet())

{'d': [2, 2, 4, 3, 4],
 'e': [3, 4, 4, 3, 6, 6, 3, 6, 4, 6, 6, 6, 4, 6, 3, 4, 6, 4, 4, 4],
 'ks': [3, 7, 7, 7, 3, 3, 5, 7, 3, 7, 5, 7, 5, 5, 3, 7, 5, 7, 7, 7]}


In [36]:
active_subnet = dense_base_dyn_net.get_active_subnet()

In [37]:
print(active_subnet.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv3_RELU_O24_BN, None)
(7x7_MBConv4_RELU_O24_BN, Identity)
(SE_3x3_MBConv6_RELU_O40_BN, None)
(SE_3x3_MBConv6_RELU_O40_BN, Identity)
(3x3_MBConv4_H_SWISH_O80_BN, None)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(5x5_MBConv6_H_SWISH_O80_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(SE_5x5_MBConv4_H_SWISH_O112_BN, None)
(SE_5x5_MBConv6_H_SWISH_O112_BN, Identity)
(SE_3x3_MBConv3_H_SWISH_O112_BN, Identity)
(SE_5x5_MBConv6_H_SWISH_O160_BN, None)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [38]:
pprint(dense_base_dyn_net.get_active_net_config())

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 3,
                                       'in_channels': 16,
                                       'kernel_size': 3,
                                       'mid_channels': 48,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'in_channels': 24,
                                       'kernel_size': 7,
                                       'mid_channels': 96,
                            

In [39]:
pprint(active_subnet.config)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 3,
                                       'groups': None,
                                       'in_channels': 16,
                                       'kernel_size': 3,
                                       'mid_channels': 48,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'groups': None,
                                       'in_channels': 24,
                                  

In [40]:
print(active_subnet)

SE_D_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticDenseBodyStage(
      (layers): ModuleList(
   

In [41]:
dense_base_dyn_net.set_max_net()

In [42]:
print(dense_base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
(SE(O40, E6.0, K7), None)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
((O80, E6.0, K7), None)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
(SE(O112, E6.0, K7), None)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E6.0, K7), None)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


### SE_P_OFAMobileNetV3

In [3]:
sep_net = SE_P_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
            net_width_list=net_width_list
)

In [4]:
print(sep_net.name)

SE_P_OFAMobileNetV3


In [5]:
print(sep_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [6]:
print(sep_net.head_stage)

StaticHeadStage(
  (first_conv): ConvLayer(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): Hswish()
  )
  (first_layer): ResidualBlock(
    (conv): MBConvLayer(
      (depth_conv): Sequential(
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (point_linear): Sequential(
        (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (shortcut): IdentityLayer()
  )
)


In [8]:
print(sep_net.exit_stage)

StaticExitStage(
  (final_expand_layer): ConvLayer(
    (conv): Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): Hswish()
  )
  (global_avg_pooling): MyGlobalAvgPool2d(keep_dim=True)
  (feature_mix_layer): ConvLayer(
    (conv): Conv2d(960, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (act): Hswish()
  )
  (classifier): LinearLayer(
    (dropout): Dropout(p=0.1, inplace=True)
    (linear): Linear(in_features=1280, out_features=200, bias=True)
  )
)


In [9]:
sep_net.get_bn_param()

{'momentum': 0.1, 'eps': 1e-05, 'ws_eps': None}

In [10]:
sep_net.set_bn_param(momentum=0.4,eps=2)

In [11]:
sep_net.get_bn_param()

{'momentum': 0.4, 'eps': 2, 'ws_eps': None}

In [12]:
print(sep_net)

SE_P_OFAMobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): DynamicParallelBodyStage(
      (layers_blocks): ModuleList(
 

In [13]:
net_cfg = sep_net.config

In [14]:
pprint(net_cfg)

{'bn': {'eps': 2, 'momentum': 0.4, 'ws_eps': None},
 'body_stages': [{'layers_activation_status': [[True, True, True],
                                               [True, True, True],
                                               [True, True, True],
                                               [True, True, True]],
                  'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio_list': [3, 4, 6],
                                               'in_channel_list': [16],
                                               'kernel_size_list': [3, 5, 7],
                                               'name': 'DynamicMBConvLayer',
                                               'out_channel_list': [24],
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortc

In [15]:
print(sep_net.width_mult)

1.0


In [16]:
print(sep_net.ks_list)

[3, 5, 7]


In [18]:
pprint(sep_net.sample_active_subnet())

{'d': [4, 4, 3, 4, 2],
 'e': [3, 6, 6, 6, 6, 6, 4, 4, 6, 4, 4, 4, 3, 4, 3, 4, 6, 4, 3, 3],
 'ks': [7, 3, 5, 3, 7, 5, 3, 3, 3, 5, 5, 7, 5, 3, 7, 3, 3, 7, 7, 5],
 'nw': [6, 3, 5, 3, 2, 2, 5, 3, 2, 3, 7, 6, 6, 2, 6, 1, 7, 4, 2, 1]}


In [19]:
active_subnet_cfg = sep_net.get_active_net_config()
pprint(active_subnet_cfg)

{'bn': {'eps': 2, 'momentum': 0.4, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 3,
                                               'in_channels': 16,
                                               'kernel_size': 7,
                                               'mid_channels': 48,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None},
                                     {'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
                                      'dropout_rate': 0,
              

In [20]:
active_subnet = sep_net.get_active_subnet()
pprint(active_subnet.config)

{'bn': {'eps': 2, 'momentum': 0.4, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 3,
                                               'groups': None,
                                               'in_channels': 16,
                                               'kernel_size': 7,
                                               'mid_channels': 48,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None},
                                     {'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
        

In [21]:
print(active_subnet)

SE_P_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=2, momentum=0.4, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticParallelBodyStage(
      (layers_blocks): ModuleList(
     

In [22]:
print(active_subnet.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv3_RELU_O24_BN, None)--1x1_Conv_O24_RELU_BN--
1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(5x5_MBConv6_RELU_O24_BN, Identity)--TransformationLayer24to24--
1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
1x1_Conv_O40_RELU_BN--
1x1_Conv_O40_RELU_BN--
(SE_3x3_MBConv4_RELU_O40_BN, Identity)--TransformationLayer40to40--
1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
1x1_Conv_O80_H_SWISH_BN--
1x1_Conv_O80_H_SWISH_BN--TransformationLayer80to80--
(5x5_MBConv4_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer80to80--
(SE_5x5_MBConv3_H_SWISH_O112_BN, None)--1x1_Conv_O112_H_SWISH_BN--
1x1_Conv_O112_H_SWISH_BN--
(SE_7x7_MBConv3_H_SWISH_O112_BN, Identity)--1x1_Conv_O112_H_SWISH_BN--
TransformationLayer112to112--
(SE_3x3_MBConv6_H_SWISH_O160_BN, None)--1x1_Conv_O160_H_SWISH_BN--TransformationLayer112to160--
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)--
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep

In [23]:
print(sep_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E3.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- 
1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K5), Identity) -- TransformationLayer24to24 -- 
1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
1x1_Conv_O40_RELU_BN -- 
1x1_Conv_O40_RELU_BN -- 
(SE(O40, E4.0, K3), Identity) -- TransformationLayer40to40 -- 
1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
1x1_Conv_O80_H_SWISH_BN -- 
1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E4.0, K5), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
(SE(O112, E3.0, K5), None) -- 1x1_Conv_O112_H_SWISH_BN -- 
1x1_Conv_O112_H_SWISH_BN -- 
(SE(O112, E3.0, K7), Identity) -- 1x1_Conv_O112_H_SWISH_BN -- 
TransformationLayer112to112 -- 
(SE(O160, E6.0, K3), None) -- 1x1_Conv_O160_H_SWISH_BN -- TransformationLayer112to160 -- 
(SE(O160, E4.0, K7), Identity) -- 
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O128

In [24]:
act_subnet = sep_net.get_active_subnet()

In [25]:
print(act_subnet.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv3_RELU_O24_BN, None)--1x1_Conv_O24_RELU_BN--
1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(5x5_MBConv6_RELU_O24_BN, Identity)--TransformationLayer24to24--
1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
1x1_Conv_O40_RELU_BN--
1x1_Conv_O40_RELU_BN--
(SE_3x3_MBConv4_RELU_O40_BN, Identity)--TransformationLayer40to40--
1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
1x1_Conv_O80_H_SWISH_BN--
1x1_Conv_O80_H_SWISH_BN--TransformationLayer80to80--
(5x5_MBConv4_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer80to80--
(SE_5x5_MBConv3_H_SWISH_O112_BN, None)--1x1_Conv_O112_H_SWISH_BN--
1x1_Conv_O112_H_SWISH_BN--
(SE_7x7_MBConv3_H_SWISH_O112_BN, Identity)--1x1_Conv_O112_H_SWISH_BN--
TransformationLayer112to112--
(SE_3x3_MBConv6_H_SWISH_O160_BN, None)--1x1_Conv_O160_H_SWISH_BN--TransformationLayer112to160--
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)--
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep

In [26]:
ks=3
e=[3,3,3,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[2,3,4,4,3]
nw=[4,5,6,7, 1,6,5,3, 7,2,1,4, 5,4,3,1, 7,6,5,4]
sep_net.set_active_subnet(ks=ks,e=e,d=d, nw=nw)

In [27]:
activated = sep_net.get_active_subnet()
print(activated.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv3_RELU_O24_BN, None)--
(3x3_MBConv3_RELU_O24_BN, Identity)--TransformationLayer24to24--
TransformationLayer24to40--
(SE_3x3_MBConv4_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--
(SE_3x3_MBConv4_RELU_O40_BN, Identity)--TransformationLayer40to40--
(3x3_MBConv3_H_SWISH_O80_BN, None)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer40to80--
1x1_Conv_O80_H_SWISH_BN--
TransformationLayer80to80--
(3x3_MBConv6_H_SWISH_O80_BN, Identity)--
(SE_3x3_MBConv6_H_SWISH_O112_BN, None)--TransformationLayer80to112--
(SE_3x3_MBConv3_H_SWISH_O112_BN, Identity)--
1x1_Conv_O112_H_SWISH_BN--TransformationLayer112to112--
TransformationLayer112to112--
(SE_3x3_MBConv6_H_SWISH_O160_BN, None)--1x1_Conv_O160_H_SWISH_BN--TransformationLayer112to160--
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)--1x1_Conv_O160_H_SWISH_BN--
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)--TransformationLayer160to160--
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)

In [66]:
ks=[3,5,7,5, 5,7,3,5, 5,3,5,7, 7,3,3,5, 7,5,5,7]
e=[3,3,3,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[2,3,4,4,3]
nw=[4,5,6,7, 1,6,5,3, 7,2,1,4, 5,4,3,1, 7,6,5,4]
sep_net.set_active_subnet(ks=ks,e=e,d=d, nw=nw)

In [67]:
activated = sep_net.get_active_subnet()
print(activated.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv3_RELU_O24_BN, None)--
(5x5_MBConv3_RELU_O24_BN, Identity)--ATTENTIONCONV_24--
ATTENTIONCONV_40--
(SE_7x7_MBConv4_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--
(SE_3x3_MBConv4_RELU_O40_BN, Identity)--ATTENTIONCONV_40--
(5x5_MBConv3_H_SWISH_O80_BN, None)--1x1_Conv_O80_H_SWISH_BN--ATTENTIONCONV_80--
1x1_Conv_O80_H_SWISH_BN--
ATTENTIONCONV_80--
(7x7_MBConv6_H_SWISH_O80_BN, Identity)--
(SE_7x7_MBConv6_H_SWISH_O112_BN, None)--ATTENTIONCONV_112--
(SE_3x3_MBConv3_H_SWISH_O112_BN, Identity)--
1x1_Conv_O112_H_SWISH_BN--ATTENTIONCONV_112--
ATTENTIONCONV_112--
(SE_7x7_MBConv6_H_SWISH_O160_BN, None)--1x1_Conv_O160_H_SWISH_BN--ATTENTIONCONV_160--
(SE_5x5_MBConv6_H_SWISH_O160_BN, Identity)--1x1_Conv_O160_H_SWISH_BN--
(SE_5x5_MBConv6_H_SWISH_O160_BN, Identity)--ATTENTIONCONV_160--
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [4]:
sep_net.set_max_net()
print(sep_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [5]:
activated = sep_net.get_active_subnet()
print(activated.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, None)--1x1_Conv_O24_RELU_BN--TransformationLayer16to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(SE_7x7_MBConv6_RELU_O40_BN, None)--1x1_Conv_O40_RELU_BN--TransformationLayer24to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(7x7_MBConv6_H_SWISH_O80_BN, None)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer40to80--
(7x7_MBConv6_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer80to80--
(7x7_MBConv6_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWI

In [6]:
pprint(activated.config)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 6,
                                               'groups': None,
                                               'in_channels': 16,
                                               'kernel_size': 7,
                                               'mid_channels': 96,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None},
                                     {'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
    

In [7]:
pprint(activated)

SE_P_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticParallelBodyStage(
      (layers_blocks): Modul

In [8]:
x = torch.rand((1,3,64,64))
x = activated(x)

### SE_DP_OFAMobileNetV3

In [3]:
sedp_net = SE_DP_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
            net_width_list=net_width_list
)

In [4]:
print(sedp_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [5]:
print(sedp_net)

SE_DP_OFAMobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): DynamicDenseParallelBodyStage(
      (layers_bloc

In [6]:
net_cfg = sedp_net.config

In [7]:
pprint(net_cfg)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_activation_status': [[True, True, True],
                                               [True, True, True],
                                               [True, True, True],
                                               [True, True, True]],
                  'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio_list': [3, 4, 6],
                                               'in_channel_list': [16],
                                               'kernel_size_list': [3, 5, 7],
                                               'name': 'DynamicMBConvLayer',
                                               'out_channel_list': [24],
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'sh

In [8]:
print(sedp_net.width_mult)

1.0


In [9]:
print(sedp_net.ks_list)

[3, 5, 7]


In [10]:
pprint(sedp_net.sample_active_subnet())

{'d': [3, 2, 3, 4, 4],
 'e': [3, 6, 3, 6, 4, 4, 6, 6, 3, 3, 4, 6, 6, 6, 6, 6, 4, 6, 6, 3],
 'ks': [5, 7, 7, 3, 5, 7, 7, 7, 3, 3, 7, 3, 7, 5, 3, 7, 5, 7, 3, 3],
 'nw': [2, 6, 2, 3, 3, 2, 2, 3, 2, 2, 5, 2, 4, 1, 7, 6, 2, 6, 1, 1]}


In [11]:
active_subnet_cfg = sedp_net.get_active_net_config()
pprint(active_subnet_cfg)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
                                      'dropout_rate': 0,
                                      'groups': 1,
                                      'has_shuffle': False,
                                      'in_channels': 16,
                                      'kernel_size': 1,
                                      'name': 'ConvLayer',
                                      'ops_order': 'weight_bn_act',
                                      'out_channels': 24,
                                      'stride': 2,
                                      'use_bn': True,
                                      'use_se': False}],
                                    [{'conv': {'act_func': 'relu',
                                               'expand_ratio': 6,
                   

In [12]:
active_subnet = sedp_net.get_active_subnet()
pprint(active_subnet.config)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
                                      'dropout_rate': 0,
                                      'groups': 1,
                                      'has_shuffle': False,
                                      'in_channels': 16,
                                      'kernel_size': 1,
                                      'name': 'ConvLayer',
                                      'ops_order': 'weight_bn_act',
                                      'out_channels': 24,
                                      'stride': 2,
                                      'use_bn': True,
                                      'use_se': False}],
                                    [{'conv': {'act_func': 'relu',
                                               'expand_ratio': 6,
                   

In [13]:
print(active_subnet)

SE_DP_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticDenseParallelBodyStage(
      (layers_blocks):

In [15]:
print(active_subnet.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
1x1_Conv_O24_RELU_BN--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--
1x1_Conv_O24_RELU_BN--
1x1_Conv_O40_RELU_BN--TransformationLayer24to40--
1x1_Conv_O40_RELU_BN--
1x1_Conv_O80_H_SWISH_BN--
1x1_Conv_O80_H_SWISH_BN--
(7x7_MBConv4_H_SWISH_O80_BN, Identity)--TransformationLayer80to80--
(SE_7x7_MBConv6_H_SWISH_O112_BN, None)--
TransformationLayer112to112--
(SE_3x3_MBConv6_H_SWISH_O112_BN, Identity)--1x1_Conv_O112_H_SWISH_BN--TransformationLayer112to112--
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)--1x1_Conv_O112_H_SWISH_BN--
1x1_Conv_O160_H_SWISH_BN--
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)--1x1_Conv_O160_H_SWISH_BN--
TransformationLayer160to160--
TransformationLayer160to160--
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [16]:
print(sedp_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
1x1_Conv_O24_RELU_BN -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- 
1x1_Conv_O24_RELU_BN -- 
1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
1x1_Conv_O40_RELU_BN -- 
1x1_Conv_O80_H_SWISH_BN -- 
1x1_Conv_O80_H_SWISH_BN -- 
((O80, E4.0, K7), Identity) -- TransformationLayer80to80 -- 
(SE(O112, E6.0, K7), None) -- 
TransformationLayer112to112 -- 
(SE(O112, E6.0, K3), Identity) -- 1x1_Conv_O112_H_SWISH_BN -- TransformationLayer112to112 -- 
(SE(O112, E6.0, K7), Identity) -- 1x1_Conv_O112_H_SWISH_BN -- 
1x1_Conv_O160_H_SWISH_BN -- 
(SE(O160, E6.0, K7), Identity) -- 1x1_Conv_O160_H_SWISH_BN -- 
TransformationLayer160to160 -- 
TransformationLayer160to160 -- 
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [17]:
act_subnet = sedp_net.get_active_subnet()

In [18]:
print(act_subnet.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
1x1_Conv_O24_RELU_BN--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--
1x1_Conv_O24_RELU_BN--
1x1_Conv_O40_RELU_BN--TransformationLayer24to40--
1x1_Conv_O40_RELU_BN--
1x1_Conv_O80_H_SWISH_BN--
1x1_Conv_O80_H_SWISH_BN--
(7x7_MBConv4_H_SWISH_O80_BN, Identity)--TransformationLayer80to80--
(SE_7x7_MBConv6_H_SWISH_O112_BN, None)--
TransformationLayer112to112--
(SE_3x3_MBConv6_H_SWISH_O112_BN, Identity)--1x1_Conv_O112_H_SWISH_BN--TransformationLayer112to112--
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)--1x1_Conv_O112_H_SWISH_BN--
1x1_Conv_O160_H_SWISH_BN--
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)--1x1_Conv_O160_H_SWISH_BN--
TransformationLayer160to160--
TransformationLayer160to160--
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [19]:
ks=3
e=[3,3,3,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[2,3,4,4,3]
nw=[4,5,6,7, 1,6,5,3, 7,2,1,4, 5,4,3,1, 7,6,5,4]
sedp_net.set_active_subnet(ks=ks,e=e,d=d, nw=nw)

In [20]:
activated = sedp_net.get_active_subnet()
print(activated.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv3_RELU_O24_BN, None)--
(3x3_MBConv3_RELU_O24_BN, Identity)--TransformationLayer24to24--
TransformationLayer24to40--
(SE_3x3_MBConv4_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--
(SE_3x3_MBConv4_RELU_O40_BN, Identity)--TransformationLayer40to40--
(3x3_MBConv3_H_SWISH_O80_BN, None)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer40to80--
1x1_Conv_O80_H_SWISH_BN--
TransformationLayer80to80--
(3x3_MBConv6_H_SWISH_O80_BN, Identity)--
(SE_3x3_MBConv6_H_SWISH_O112_BN, None)--TransformationLayer80to112--
(SE_3x3_MBConv3_H_SWISH_O112_BN, Identity)--
1x1_Conv_O112_H_SWISH_BN--TransformationLayer112to112--
TransformationLayer112to112--
(SE_3x3_MBConv6_H_SWISH_O160_BN, None)--1x1_Conv_O160_H_SWISH_BN--TransformationLayer112to160--
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)--1x1_Conv_O160_H_SWISH_BN--
(SE_3x3_MBConv6_H_SWISH_O160_BN, Identity)--TransformationLayer160to160--
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)

In [21]:
sedp_net.set_max_net()
print(sedp_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [22]:
activated = sedp_net.get_active_subnet()
print(activated.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, None)--1x1_Conv_O24_RELU_BN--TransformationLayer16to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(SE_7x7_MBConv6_RELU_O40_BN, None)--1x1_Conv_O40_RELU_BN--TransformationLayer24to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(7x7_MBConv6_H_SWISH_O80_BN, None)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer40to80--
(7x7_MBConv6_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer80to80--
(7x7_MBConv6_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWI

In [23]:
pprint(activated.config)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 6,
                                               'groups': None,
                                               'in_channels': 16,
                                               'kernel_size': 7,
                                               'mid_channels': 96,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None},
                                     {'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
    

In [24]:
pprint(activated)

SE_DP_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticDenseParallelBodyStage(
      (layers_blocks):

In [25]:
x = torch.rand((1,3,64,64))
x = activated(x)

In [10]:
ks=3
e=[3,3,3,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[2,3,4,4,3]
nw=[4,5,6,7, 1,6,5,3, 7,2,1,4, 5,4,3,1, 7,6,5,4]
sedp_net.set_active_subnet(ks=ks,e=e,d=d, nw=nw)

In [11]:
pprint(sedp_net.get_active_net_config())

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 3,
                                               'in_channels': 16,
                                               'kernel_size': 3,
                                               'mid_channels': 48,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None}],
                                    [{'conv': {'act_func': 'relu',
                                               'expand_ratio': 3,
                                               'in_channels': 24,
                               

In [14]:
print(sedp_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E3.0, K3), None) -- 
((O24, E3.0, K3), Identity) -- TransformationLayer24to24 -- 
TransformationLayer24to40 -- 
(SE(O40, E4.0, K3), Identity) -- 1x1_Conv_O40_RELU_BN -- 
(SE(O40, E4.0, K3), Identity) -- TransformationLayer40to40 -- 
((O80, E3.0, K3), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
1x1_Conv_O80_H_SWISH_BN -- 
TransformationLayer80to80 -- 
((O80, E6.0, K3), Identity) -- 
(SE(O112, E6.0, K3), None) -- TransformationLayer80to112 -- 
(SE(O112, E3.0, K3), Identity) -- 
1x1_Conv_O112_H_SWISH_BN -- TransformationLayer112to112 -- 
TransformationLayer112to112 -- 
(SE(O160, E6.0, K3), None) -- 1x1_Conv_O160_H_SWISH_BN -- TransformationLayer112to160 -- 
(SE(O160, E6.0, K3), Identity) -- 1x1_Conv_O160_H_SWISH_BN -- 
(SE(O160, E6.0, K3), Identity) -- TransformationLayer160to160 -- 
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [15]:
activated = sedp_net.get_active_subnet()
x = torch.rand((1,3,64,64))
x = activated(x)

# Early_Exit_OFAMobileNetV3

### EE_B_OFAMobileNetV3

In [73]:
ee_base_dyn_net = EE_B_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
            net_depth_list = net_depth_list
)

In [74]:
print(ee_base_dyn_net.name)

EE_B_OFAMobileNetV3


In [75]:
print(ee_base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
(SE(O40, E6.0, K7), None)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
((O80, E6.0, K7), None)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
(SE(O112, E6.0, K7), None)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E6.0, K7), None)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [76]:
print(ee_base_dyn_net.exit_stages)

ModuleList(
  (0): StaticExitStage(
    (final_expand_layer): ConvLayer(
      (conv): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (global_avg_pooling): MyGlobalAvgPool2d(keep_dim=True)
    (feature_mix_layer): ConvLayer(
      (conv): Conv2d(144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act): Hswish()
    )
    (classifier): LinearLayer(
      (dropout): Dropout(p=0.1, inplace=True)
      (linear): Linear(in_features=192, out_features=200, bias=True)
    )
  )
  (1): StaticExitStage(
    (final_expand_layer): ConvLayer(
      (conv): Conv2d(40, 240, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (global_avg_pooling): MyGlobalAvgPool2d(keep_dim=True)
    (feature_mix_layer): ConvLayer(
      (conv): Conv

In [77]:
print(ee_base_dyn_net)

EE_B_OFAMobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): DynamicBodyStage(
      (layers): ModuleList(
    

In [78]:
net_cfg_eeb = ee_base_dyn_net.config
pprint(net_cfg_eeb)

{'active_exit': 5,
 'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [16],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'DynamicMBConvLayer',
                                       'out_channel_list': [24],
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [24],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'Dy

In [79]:
for _ in range(5):
    pprint(ee_base_dyn_net.sample_active_subnet())
    print(ee_base_dyn_net.get_active_exit())
    sampled = ee_base_dyn_net.get_active_subnet()
    print(sampled.module_str)
    print("\n")

{'d': [2, 3, 3, 2, 4],
 'e': [4, 6, 3, 6, 6, 3, 3, 6, 6, 3, 4, 6, 6, 4, 4, 3, 4, 6, 3, 4],
 'ks': [5, 3, 7, 3, 7, 5, 5, 5, 7, 5, 5, 3, 3, 3, 3, 3, 5, 7, 5, 7],
 'nd': 5}
5
3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(5x5_MBConv4_RELU_O24_BN, None)
(3x3_MBConv6_RELU_O24_BN, Identity)
(SE_7x7_MBConv6_RELU_O40_BN, None)
(SE_5x5_MBConv3_RELU_O40_BN, Identity)
(SE_5x5_MBConv3_RELU_O40_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, None)
(5x5_MBConv3_H_SWISH_O80_BN, Identity)
(5x5_MBConv4_H_SWISH_O80_BN, Identity)
(SE_3x3_MBConv6_H_SWISH_O112_BN, None)
(SE_3x3_MBConv4_H_SWISH_O112_BN, Identity)
(SE_5x5_MBConv4_H_SWISH_O160_BN, None)
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)
(SE_5x5_MBConv3_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv4_H_SWISH_O160_BN, Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


{'d': [3, 3, 4, 3, 4],
 'e': [4, 4, 3, 3, 4, 6, 3, 6, 4, 3, 6, 6, 4, 3, 6, 3, 6, 6, 6, 4],
 'ks': [3, 7, 7, 5, 3, 5, 5, 3, 5,

In [80]:
print(sampled)

SE_B_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticBodyStage(
      (layers): ModuleList(
        

In [81]:
pprint(sampled.config)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'groups': None,
                                       'in_channels': 16,
                                       'kernel_size': 7,
                                       'mid_channels': 64,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 6,
                                       'groups': None,
                                       'in_channels': 24,
                                  

In [82]:
pprint(ee_base_dyn_net.get_active_net_config())

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'in_channels': 16,
                                       'kernel_size': 7,
                                       'mid_channels': 64,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 6,
                                       'in_channels': 24,
                                       'kernel_size': 7,
                                       'mid_channels': 144,
                           

In [83]:
ee_base_dyn_net.set_max_net()

In [84]:
print(ee_base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
(SE(O40, E6.0, K7), None)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
((O80, E6.0, K7), None)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
(SE(O112, E6.0, K7), None)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E6.0, K7), None)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [85]:
print(ee_base_dyn_net.get_active_subnet().module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, None)
(7x7_MBConv6_RELU_O24_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, Identity)
(SE_7x7_MBConv6_RELU_O40_BN, None)
(SE_7x7_MBConv6_RELU_O40_BN, Identity)
(SE_7x7_MBConv6_RELU_O40_BN, Identity)
(SE_7x7_MBConv6_RELU_O40_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, None)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(7x7_MBConv6_H_SWISH_O80_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, None)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O112_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O160_BN, None)
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)
(SE_7x7_MBConv6_H_SWISH_O160_BN, Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


In [86]:
pprint(ee_base_dyn_net.get_active_net_config())

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 6,
                                       'in_channels': 16,
                                       'kernel_size': 7,
                                       'mid_channels': 96,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 6,
                                       'in_channels': 24,
                                       'kernel_size': 7,
                                       'mid_channels': 144,
                           

In [35]:
ks=7
e=[3,3,3,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[2,3,4,4,3]
nd = 2
ee_base_dyn_net.set_active_subnet(ks=ks,e=e,d=d, nd=nd)

NameError: name 'ee_base_dyn_net' is not defined

In [88]:
print(ee_base_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E3.0, K7), None)
((O24, E3.0, K7), Identity)
(SE(O40, E4.0, K7), None)
(SE(O40, E4.0, K7), Identity)
(SE(O40, E4.0, K7), Identity)
1x1_Conv_O240_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O320_H_SWISH
320x200_Linear


### EE_D_OFAMobileNetV3

In [89]:
dense_ee_dyn_net = EE_D_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
            net_depth_list= net_depth_list
)

In [90]:
print(dense_ee_dyn_net.name)

EE_D_OFAMobileNetV3


In [91]:
net_cfg_dee = dense_ee_dyn_net.config
pprint(net_cfg_dee)

{'active_exit': 5,
 'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [16],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'DynamicMBConvLayer',
                                       'out_channel_list': [24],
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio_list': [3, 4, 6],
                                       'in_channel_list': [24],
                                       'kernel_size_list': [3, 5, 7],
                                       'name': 'Dy

In [92]:
for _ in range(2):
    pprint(dense_ee_dyn_net.sample_active_subnet())
    print(dense_ee_dyn_net.get_active_exit())
    sampled = dense_ee_dyn_net.get_active_subnet()
    print(sampled.module_str)
    print("\n")

{'d': [2, 2, 2, 4, 3],
 'e': [4, 4, 3, 6, 4, 3, 4, 3, 6, 3, 3, 6, 4, 3, 4, 6, 6, 4, 4, 6],
 'ks': [3, 3, 7, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 5, 5, 7, 3, 7, 7],
 'nd': 2}
2
3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv4_RELU_O24_BN, None)
(3x3_MBConv4_RELU_O24_BN, Identity)
(SE_3x3_MBConv4_RELU_O40_BN, None)
(SE_3x3_MBConv3_RELU_O40_BN, Identity)
1x1_Conv_O240_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O320_H_SWISH
320x200_Linear


{'d': [3, 4, 3, 3, 3],
 'e': [4, 3, 3, 4, 4, 6, 6, 6, 6, 4, 4, 6, 3, 3, 6, 4, 6, 3, 3, 4],
 'ks': [5, 7, 3, 5, 3, 7, 7, 3, 7, 3, 5, 5, 5, 7, 3, 5, 7, 3, 7, 7],
 'nd': 1}
1
3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(5x5_MBConv4_RELU_O24_BN, None)
(7x7_MBConv3_RELU_O24_BN, Identity)
(3x3_MBConv3_RELU_O24_BN, Identity)
1x1_Conv_O144_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O192_H_SWISH
192x200_Linear




In [93]:
print(sampled)

SE_D_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticDenseBodyStage(
      (layers): ModuleList(
   

In [94]:
pprint(sampled.config)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'groups': None,
                                       'in_channels': 16,
                                       'kernel_size': 5,
                                       'mid_channels': 64,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 3,
                                       'groups': None,
                                       'in_channels': 24,
                                  

In [95]:
pprint(dense_ee_dyn_net.get_active_net_config())

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers': [{'conv': {'act_func': 'relu',
                                       'expand_ratio': 4,
                                       'in_channels': 16,
                                       'kernel_size': 5,
                                       'mid_channels': 64,
                                       'name': 'MBConvLayer',
                                       'out_channels': 24,
                                       'stride': 2,
                                       'use_se': False},
                              'name': 'ResidualBlock',
                              'shortcut': None},
                             {'conv': {'act_func': 'relu',
                                       'expand_ratio': 3,
                                       'in_channels': 24,
                                       'kernel_size': 7,
                                       'mid_channels': 72,
                            

In [96]:
dense_ee_dyn_net.set_max_net()

In [97]:
print(dense_ee_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
((O24, E6.0, K7), Identity)
(SE(O40, E6.0, K7), None)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
(SE(O40, E6.0, K7), Identity)
((O80, E6.0, K7), None)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
((O80, E6.0, K7), Identity)
(SE(O112, E6.0, K7), None)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O112, E6.0, K7), Identity)
(SE(O160, E6.0, K7), None)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
(SE(O160, E6.0, K7), Identity)
1x1_Conv_O960_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O1280_H_SWISH
1280x200_Linear


### EE_P_OFAMobileNetV3

In [26]:
eep_dyn_net = EE_P_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
            net_depth_list = net_depth_list,
            net_width_list = net_width_list
)

In [27]:
print(eep_dyn_net.name)

EE_P_OFAMobileNetV3


In [28]:
print(eep_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [29]:
print(eep_dyn_net.exit_stages)

ModuleList(
  (0): StaticExitStage(
    (final_expand_layer): ConvLayer(
      (conv): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (global_avg_pooling): MyGlobalAvgPool2d(keep_dim=True)
    (feature_mix_layer): ConvLayer(
      (conv): Conv2d(144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act): Hswish()
    )
    (classifier): LinearLayer(
      (dropout): Dropout(p=0.1, inplace=True)
      (linear): Linear(in_features=192, out_features=200, bias=True)
    )
  )
  (1): StaticExitStage(
    (final_expand_layer): ConvLayer(
      (conv): Conv2d(40, 240, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (global_avg_pooling): MyGlobalAvgPool2d(keep_dim=True)
    (feature_mix_layer): ConvLayer(
      (conv): Conv

In [30]:
print(eep_dyn_net.body_stages)

ModuleList(
  (0): DynamicParallelBodyStage(
    (layers_blocks): ModuleList(
      (0): ModuleList(
        (0): ResidualBlock(
          (conv): DynamicMBConvLayer(
            (inverted_bottleneck): Sequential(
              (conv): DynamicConv2d(
                (conv): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
              )
              (bn): DynamicBatchNorm2d(
                (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (act): ReLU(inplace=True)
            )
            (depth_conv): Sequential(
              (conv): DynamicSeparableConv2d(
                (conv): Conv2d(96, 96, kernel_size=(7, 7), stride=(2, 2), groups=96, bias=False)
              )
              (bn): DynamicBatchNorm2d(
                (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (act): ReLU(inplace=True)
            )
            (point_linear)

In [31]:
net_cfg_eep = eep_dyn_net.config
pprint(net_cfg_eep)

{'active_exit': 5,
 'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_activation_status': [[True, True, True],
                                               [True, True, True],
                                               [True, True, True],
                                               [True, True, True]],
                  'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio_list': [3, 4, 6],
                                               'in_channel_list': [16],
                                               'kernel_size_list': [3, 5, 7],
                                               'name': 'DynamicMBConvLayer',
                                               'out_channel_list': [24],
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                      

In [32]:
for _ in range(2):
    pprint(eep_dyn_net.sample_active_subnet())
    print(eep_dyn_net.get_active_exit())
    sampled = eep_dyn_net.get_active_subnet()
    print(sampled.module_str)
    print("\n")

{'d': [3, 4, 3, 4, 4],
 'e': [4, 3, 4, 4, 6, 3, 3, 3, 3, 3, 6, 4, 4, 4, 6, 3, 3, 4, 4, 3],
 'ks': [7, 3, 7, 5, 3, 7, 5, 3, 5, 3, 5, 3, 3, 5, 7, 3, 5, 7, 7, 5],
 'nd': 1,
 'nw': [4, 7, 5, 5, 5, 1, 4, 1, 2, 2, 7, 7, 6, 4, 4, 3, 2, 5, 3, 5]}
1
3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv4_RELU_O24_BN, None)--
(3x3_MBConv3_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(7x7_MBConv4_RELU_O24_BN, Identity)--TransformationLayer24to24--
1x1_Conv_O144_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O192_H_SWISH
192x200_Linear


{'d': [2, 2, 4, 3, 2],
 'e': [3, 4, 4, 3, 6, 3, 6, 3, 3, 4, 4, 3, 6, 6, 4, 4, 4, 3, 6, 3],
 'ks': [3, 5, 7, 5, 5, 7, 7, 7, 7, 3, 5, 7, 5, 7, 3, 7, 7, 5, 5, 5],
 'nd': 3,
 'nw': [6, 3, 6, 5, 7, 1, 5, 4, 3, 2, 7, 4, 4, 1, 7, 4, 7, 6, 5, 5]}
3
3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(3x3_MBConv3_RELU_O24_BN, None)--1x1_Conv_O24_RELU_BN--
1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(SE_5x5_M

In [33]:
print(sampled)

SE_P_MobileNetV3(
  (head_stage): StaticHeadStage(
    (first_conv): ConvLayer(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Hswish()
    )
    (first_layer): ResidualBlock(
      (conv): MBConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
  )
  (body_stages): ModuleList(
    (0): StaticParallelBodyStage(
      (layers_blocks): Modul

In [34]:
pprint(sampled.config)

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 3,
                                               'groups': None,
                                               'in_channels': 16,
                                               'kernel_size': 3,
                                               'mid_channels': 48,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None},
                                     {'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
    

In [35]:
pprint(eep_dyn_net.get_active_net_config())

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 3,
                                               'in_channels': 16,
                                               'kernel_size': 3,
                                               'mid_channels': 48,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None},
                                     {'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
                                      'dropout_rate': 0,
          

In [36]:
eep_dyn_net.set_max_net()

In [37]:
print(eep_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [38]:
print(eep_dyn_net.get_active_subnet().module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
(7x7_MBConv6_RELU_O24_BN, None)--1x1_Conv_O24_RELU_BN--TransformationLayer16to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(7x7_MBConv6_RELU_O24_BN, Identity)--1x1_Conv_O24_RELU_BN--TransformationLayer24to24--
(SE_7x7_MBConv6_RELU_O40_BN, None)--1x1_Conv_O40_RELU_BN--TransformationLayer24to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(SE_7x7_MBConv6_RELU_O40_BN, Identity)--1x1_Conv_O40_RELU_BN--TransformationLayer40to40--
(7x7_MBConv6_H_SWISH_O80_BN, None)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer40to80--
(7x7_MBConv6_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWISH_BN--TransformationLayer80to80--
(7x7_MBConv6_H_SWISH_O80_BN, Identity)--1x1_Conv_O80_H_SWI

In [39]:
pprint(eep_dyn_net.get_active_net_config())

{'bn': {'eps': 1e-05, 'momentum': 0.1, 'ws_eps': None},
 'body_stages': [{'layers_blocks': [[{'conv': {'act_func': 'relu',
                                               'expand_ratio': 6,
                                               'in_channels': 16,
                                               'kernel_size': 7,
                                               'mid_channels': 96,
                                               'name': 'MBConvLayer',
                                               'out_channels': 24,
                                               'stride': 2,
                                               'use_se': False},
                                      'name': 'ResidualBlock',
                                      'shortcut': None},
                                     {'act_func': 'relu',
                                      'bias': False,
                                      'dilation': 1,
                                      'dropout_rate': 0,
          

In [40]:
ks=[3,5,7,5, 3,5,7,3, 5,7,5,3, 7,5,3,3, 7,5,3,3]
e=[3,4,6,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[4,3,4,4,3]
nw = [7,6,5,4, 3,2,1,7, 6,5,4,3, 2,1,7,6, 5,4,3,2]
nd = 2
eep_dyn_net.set_active_subnet(ks=ks,e=e,d=d, nw=nw, nd=nd)

In [41]:
print(eep_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E3.0, K3), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E4.0, K5), Identity) -- 1x1_Conv_O24_RELU_BN -- 
((O24, E6.0, K7), Identity) -- TransformationLayer24to24 -- 
((O24, E3.0, K5), Identity) -- 
1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
1x1_Conv_O40_RELU_BN -- 
TransformationLayer40to40 -- 
1x1_Conv_O240_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O320_H_SWISH
320x200_Linear


In [42]:
activated = sedp_net.get_active_subnet()
x = torch.rand((1,3,64,64))
x = activated(x)

### EE_DP_OFAMobileNetV3

In [43]:
eedp_dyn_net = EE_DP_OFAMobileNetV3(
            n_classes=n_classes,
            bn_param=bn_param,
            dropout_rate=dropout_rate,
            width_mult=width_mult,
            ks_list=ks_list,
            expand_ratio_list=expand_ratio_list,
            depth_list=depth_list,
            net_depth_list = net_depth_list,
            net_width_list = net_width_list
)

In [44]:
print(eedp_dyn_net.name)

EE_DP_OFAMobileNetV3


In [45]:
print(eedp_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [46]:
ks=[3,5,7,5, 3,5,7,3, 5,7,5,3, 7,5,3,3, 7,5,3,3]
e=[3,4,6,3, 4,4,4,4, 3,4,6,6, 6,3,4,4, 6,6,6,6]
d=[4,3,4,4,3]
nw = [7,6,5,4, 3,2,1,7, 6,5,4,3, 2,1,7,6, 5,4,3,2]
nd = 2
eedp_dyn_net.set_active_subnet(ks=ks,e=e,d=d, nw=nw, nd=nd)

In [47]:
print(eedp_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E3.0, K3), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E4.0, K5), Identity) -- 1x1_Conv_O24_RELU_BN -- 
((O24, E6.0, K7), Identity) -- TransformationLayer24to24 -- 
((O24, E3.0, K5), Identity) -- 
1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
1x1_Conv_O40_RELU_BN -- 
TransformationLayer40to40 -- 
1x1_Conv_O240_H_SWISH_BN
MyGlobalAvgPool2d(keep_dim=True)
1x1_Conv_O320_H_SWISH
320x200_Linear


In [48]:
eedp_dyn_net.set_max_net()
print(eedp_dyn_net.module_str)

3x3_Conv_O16_H_SWISH_BN
(3x3_MBConv1_RELU_O16_BN, Identity)
((O24, E6.0, K7), None) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer16to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
((O24, E6.0, K7), Identity) -- 1x1_Conv_O24_RELU_BN -- TransformationLayer24to24 -- 
(SE(O40, E6.0, K7), None) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer24to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
(SE(O40, E6.0, K7), Identity) -- 1x1_Conv_O40_RELU_BN -- TransformationLayer40to40 -- 
((O80, E6.0, K7), None) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer40to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 
((O80, E6.0, K7), Identity) -- 1x1_Conv_O80_H_SWISH_BN -- TransformationLayer80to80 -- 


In [49]:
activated = sedp_net.get_active_subnet()
x = torch.rand((1,3,64,64))
x = activated(x)